# Workflow Example with ERA5 input convertor
Here we illustrate a complete workflow example using the ERA5 convertor

## Imports

In [1]:
from pathlib import Path

import xarray as xr
import pandas as pd

import valenspy as vp #The Valenspy package
from valenspy.inputconverter_functions import _non_convertor, ERA5_to_CF

from yaml import safe_load
from typing import Union, List


In [2]:
#Import Converter - This input converter will not do anything to the data.
ic = vp.InputConverter(ERA5_to_CF)


## 1. Convert the data

### Get the filenames of ERA5 data corresponding to CORDEX variable name


In [3]:

machine = "hortense"


# User defined variable, here we look at 2m temperature. 
variable = "tas"
dataset = "ERA5"
region = "europe" # "belgium"
time_freq = "daily" ## Important to account here are the postprocessed files at daily time resolution and their naming (eg max daily temperature does not correspond to ERA5 variable name as defined!!)

### Generate the path and filename of obs file

In [4]:
# Get the current file directory and load the CORDEX variables.yml file

# files = Path(__file__).resolve().parent -- this is not working in notebook
src_path = Path("../src/valenspy") ## -- to be removed. 

with open(src_path / "ancilliary_data" / "CORDEX_variables.yml") as file:
    CORDEX_VARIABLES = safe_load(file)

# ERA5-Land has same variable defenitions as ERA5
if dataset == 'ERA5-Land':
    dataset_lookup = 'ERA5'
else: 
    dataset_lookup = dataset

with open(src_path / "ancilliary_data" / Path(dataset_lookup+"_lookup.yml")) as file:
    obs_LOOKUP = safe_load(file)

In [5]:
# this will be part of the PATH generator. 

# get path of observational datasets from path settings .yml
with open(src_path / "ancilliary_data" / Path("dataset_PATHS.yml")) as file:
    dataset_PATHS = safe_load(file)
directory = dataset_PATHS[machine][dataset]

# get ERA5 variable corresponding to the requested variable using its look-up table
obs_var = obs_LOOKUP[variable]['obs_name']

# get ERA5 subdirectory
subdirectory = Path(directory + '/' + region +'/'+time_freq+'/'+obs_LOOKUP[variable]['obs_name']+'/')

# open the EOBS file for the corresponding variable
files = list(subdirectory.glob("*-"+time_freq+"-*"+obs_var+"*.nc")) #Select all the netCDF files in the directory

In [6]:
# test of ERA5 convertor function
ERA5_ds = ic.convert_input(files) #Convert the input to the correct format
ERA5_ds


The file is NOT ValEnsPy CF compliant.
0.00% of the variables are ValEnsPy CF compliant
Unknown to ValEnsPy: ['time_bnds', 't2m']


<xarray.Dataset> Size: 6GB
Dimensions:    (time: 14975, bnds: 2, latitude: 163, longitude: 289)
Coordinates:
  * time       (time) datetime64[ns] 120kB 1979-01-01T11:30:00 ... 2019-12-31...
  * longitude  (longitude) float32 1kB -27.0 -26.75 -26.5 ... 44.5 44.75 45.0
  * latitude   (latitude) float32 652B 73.5 73.25 73.0 72.75 ... 33.5 33.25 33.0
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) datetime64[ns] 240kB dask.array<chunksize=(365, 2), meta=np.ndarray>
    t2m        (time, latitude, longitude) float64 6GB dask.array<chunksize=(362, 161, 286), meta=np.ndarray>
Attributes:
    CDI:          Climate Data Interface version 1.9.5 (http://mpimet.mpg.de/...
    Conventions:  CF-1.6
    history:      Tue Feb 08 15:58:26 2022: cdo -z zip daymean /mnt/HDS_BREGI...
    frequency:    day
    CDO:          Climate Data Operators version 1.9.5 (http://mpimet.mpg.de/...